In [2]:
from src.utils.env_export import create_json
from src.utils.khalili_env import get_env

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
cost_weight= 0.5798921379230035

for size in range(2, 6):
    env, config = get_env(size)
    create_json(f"./data/khalili_env_{size}.json", config, list(range(1, 31)), cost_weight=cost_weight)